In [ ]:
import cPickle
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Pre-process data

In [ ]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def reshape_img(img):
    img = img.reshape((3,32*32))
    v = np.zeros((32,32,3),dtype=np.uint8)
    for i in range(32):
        for j in range(32):
            v[i][j][0] = img[0][i*32+j]
            v[i][j][1] = img[1][i*32+j]
            v[i][j][2] = img[2][i*32+j]  
    return v


#### Train set

#### Test set

### Load Dataset

In [ ]:
DATASET_DIR = "dataset/"
DATASET_FILE = "train.pkl"

LABELS = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
# LABELS = ['airplane','automobile','bird','cat','deer','frog','horse','ship','truck']

dataset = pd.read_pickle(DATASET_DIR + DATASET_FILE)
dataset = dataset.sample(3000,random_state=7)
# dataset = dataset[dataset.label != 3]
# dataset = dataset[dataset.label != 5]

SUFFIX = 7

#### Train-Test split

In [ ]:
dataset_train, dataset_test = train_test_split(dataset,test_size=.2)

## Linear SVM

In [ ]:
import cv2
from skimage.feature import hog
from skimage import color
from sklearn.svm import LinearSVC,SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#### 1). Using HOG descriptors

In [ ]:
%%time
print "Computing descriptors for train set"
X_train = []
for img in dataset_train.img.values:
    X_train.append(hog(color.rgb2gray(img)))
X_train = np.array(X_train)

print "Computing descriptors for test set"
X_test = []
for img in dataset_test.img.values:
    X_test.append(hog(color.rgb2gray(img)))
X_test = np.array(X_test)

y_train = dataset_train.label.values
y_test = dataset_test.label.values

#### 2). Using Kmeans clustered BOW

##### Compute and dump Vocab

##### Load Vocab

##### Get BOW vectors

#### Init and Fit Model

In [ ]:
%%time
model = LinearSVC(multi_class='crammer_singer',verbose=1)
# model = SVC(decision_function_shape='ova',kernel='linear',verbose=True)
model.fit(X_train,y_train)

#### Evaluate

In [ ]:
score = model.score(X_test,y_test)
score

In [ ]:
f = open("output/score"+str(SUFFIX)+".txt",'w+')
f.write("%s\n\n" % str(score))
f.write(classification_report(y_test,preds,target_names=LABELS))
f.close()

#### Predict

In [ ]:
preds = model.predict(X_test)

#### Draw confusion matrix

In [ ]:
cm = confusion_matrix(preds,y_test)
df_cm = pd.DataFrame(cm, index = LABELS,
                  columns = LABELS)
sn.heatmap(df_cm,annot=True,fmt="")
plt.savefig('output/fig'+str(SUFFIX)+'.png')
plt.show()

In [ ]:
out = pd.DataFrame({'Predicted': preds,'Actual': y_test})
out.to_csv("output/preds_"+str(SUFFIX)+".csv")

In [ ]:
print classification_report(y_test,preds,target_names=LABELS)